In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab Notebooks/fasttext

/content/drive/MyDrive/Colab Notebooks/fasttext


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

# Pytorch Imports
import torch
import torch.nn as nn
import torch.utils.data
from torch.autograd import Variable

import time

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score,classification_report,accuracy_score,confusion_matrix,mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Embedding, BatchNormalization
from keras.layers.core import Dense, Activation, Dropout
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import text
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from nltk import word_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
nltk.download('punkt')

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [ ]:
train.author.unique()

array(['EAP', 'HPL', 'MWS'], dtype=object)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19579 entries, 0 to 19578
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      19579 non-null  object
 1   text    19579 non-null  object
 2   author  19579 non-null  object
dtypes: object(3)
memory usage: 459.0+ KB


In [ ]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [ ]:
np.unique(y)

array([0, 1, 2])

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [ ]:
print (xtrain.shape)
print (xvalid.shape)

(17621,)
(1958,)


In [ ]:
embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [02:18, 15900.43it/s]

Found 2196016 word vectors.


In [ ]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())


In [ ]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]


100%|██████████| 17621/17621 [00:07<00:00, 2473.17it/s]

100%|██████████| 1958/1958 [00:00<00:00, 2440.58it/s]


In [ ]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [ ]:
xtrain_glove.shape

(17621, 300)

In [ ]:
xtrain_glove[0].shape

(300,)

In [ ]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [ ]:
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [ ]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')


In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 300)               90300     
                                                                 
 dropout_18 (Dropout)        (None, 300)               0         
                                                                 
 batch_normalization_6 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_28 (Dense)            (None, 300)               90300     
                                                                 
 dropout_19 (Dropout)        (None, 300)               0         
                                                                 
 batch_normalization_7 (Batc  (None, 300)              1200      
 hNormalization)                                      

In [ ]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=128, 
          epochs=100, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
138/138 [==============================] - 1s 8ms/step - loss: 0.0571 - accuracy: 0.9793 - val_loss: 1.3099 - val_accuracy: 0.7329
Epoch 2/100
138/138 [==============================] - 1s 9ms/step - loss: 0.0599 - accuracy: 0.9789 - val_loss: 1.2922 - val_accuracy: 0.7482
Epoch 3/100
138/138 [==============================] - 1s 8ms/step - loss: 0.0665 - accuracy: 0.9773 - val_loss: 1.2877 - val_accuracy: 0.7303
Epoch 4/100
138/138 [==============================] - 1s 5ms/step - loss: 0.0648 - accuracy: 0.9772 - val_loss: 1.2941 - val_accuracy: 0.7360
Epoch 5/100
138/138 [==============================] - 1s 5ms/step - loss: 0.0561 - accuracy: 0.9809 - val_loss: 1.2717 - val_accuracy: 0.7400
Epoch 6/100
138/138 [==============================] - 1s 5ms/step - loss: 0.0568 - accuracy: 0.9805 - val_loss: 1.2596 - val_accuracy: 0.7329
Epoch 7/100
138/138 [==============================] - 1s 5ms/step - loss: 0.0561 - accuracy: 0.9802 - val_loss: 1.3142 - val_accuracy: 0.7370

In [ ]:
model.evaluate(xvalid_glove_scl, yvalid_enc)

62/62 [==============================] - 0s 3ms/step - loss: 0.6556 - accuracy: 0.7431


[0.6556088328361511, 0.7431052327156067]

In [ ]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
len(xtrain_seq[0])

24

In [ ]:
len(xtrain_seq[1])

17

In [ ]:
xtrain_pad[0].shape

(70,)

In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector =  embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


100%|██████████| 25943/25943 [00:00<00:00, 263337.33it/s]


In [ ]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')


In [ ]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 70, 300)           7783200   
                                                                 
 spatial_dropout1d_6 (Spatia  (None, 70, 300)          0         
 lDropout1D)                                                     
                                                                 
 lstm_5 (LSTM)               (None, 100)               160400    
                                                                 
 dense_30 (Dense)            (None, 1024)              103424    
                                                                 
 dropout_20 (Dropout)        (None, 1024)              0         
                                                                 
 dense_31 (Dense)            (None, 1024)              1049600   
                                                     

In [ ]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 14s 319ms/step - loss: 1.0599 - accuracy: 0.4341 - val_loss: 0.9224 - val_accuracy: 0.5822
Epoch 2/100
35/35 [==============================] - 11s 315ms/step - loss: 0.8942 - accuracy: 0.5901 - val_loss: 0.7474 - val_accuracy: 0.6936
Epoch 3/100
35/35 [==============================] - 11s 308ms/step - loss: 0.8166 - accuracy: 0.6439 - val_loss: 0.6989 - val_accuracy: 0.7094
Epoch 4/100
35/35 [==============================] - 11s 314ms/step - loss: 0.7923 - accuracy: 0.6584 - val_loss: 0.6974 - val_accuracy: 0.7201
Epoch 5/100
35/35 [==============================] - 14s 406ms/step - loss: 0.7635 - accuracy: 0.6744 - val_loss: 0.6658 - val_accuracy: 0.7324
Epoch 6/100
35/35 [==============================] - 11s 310ms/step - loss: 0.7451 - accuracy: 0.6829 - val_loss: 0.6454 - val_accuracy: 0.7360
Epoch 7/100
35/35 [==============================] - 11s 315ms/step - loss: 0.7212 - accuracy: 0.6941 - val_loss: 0.6355 - val_accuracy:

In [ ]:
model.evaluate(xvalid_pad, yvalid_enc)

62/62 [==============================] - 1s 23ms/step - loss: 0.5164 - accuracy: 0.8003


[0.5164366364479065, 0.8003064393997192]

In [ ]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 70, 300)           7783200   
                                                                 
 spatial_dropout1d_7 (Spatia  (None, 70, 300)          0         
 lDropout1D)                                                     
                                                                 
 gru_2 (GRU)                 (None, 70, 300)           541800    
                                                                 
 gru_3 (GRU)                 (None, 300)               541800    
                                                                 
 dense_33 (Dense)            (None, 1024)              308224    
                                                                 
 dropout_22 (Dropout)        (None, 1024)              0         
                                                     

In [ ]:
# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 44s 627ms/step - loss: 1.0539 - accuracy: 0.4417 - val_loss: 0.8959 - val_accuracy: 0.5889
Epoch 2/100
35/35 [==============================] - 22s 627ms/step - loss: 0.8982 - accuracy: 0.5957 - val_loss: 0.7655 - val_accuracy: 0.6782
Epoch 3/100
35/35 [==============================] - 21s 607ms/step - loss: 0.8273 - accuracy: 0.6403 - val_loss: 0.7293 - val_accuracy: 0.7160
Epoch 4/100
35/35 [==============================] - 23s 651ms/step - loss: 0.7976 - accuracy: 0.6580 - val_loss: 0.7194 - val_accuracy: 0.7074
Epoch 5/100
35/35 [==============================] - 22s 618ms/step - loss: 0.7708 - accuracy: 0.6721 - val_loss: 0.7097 - val_accuracy: 0.7160
Epoch 6/100
35/35 [==============================] - 21s 609ms/step - loss: 0.7529 - accuracy: 0.6794 - val_loss: 0.6982 - val_accuracy: 0.7120
Epoch 7/100
35/35 [==============================] - 21s 610ms/step - loss: 0.7248 - accuracy: 0.6968 - val_loss: 0.6573 - val_accuracy:

In [ ]:
model.evaluate(xvalid_pad, yvalid_enc)

62/62 [==============================] - 5s 83ms/step - loss: 0.4654 - accuracy: 0.8151


[0.4653916358947754, 0.8151174783706665]